# Database normalisation of olympic data. 

Datasets are often denormalised relational databases into large _flat tables_.  It is worth considering reversing this data engineering task, so that we understand better the structure of the data, prepare better data for model fitting for advanced statistical methodologies and machine learning algorithms.

It is worth eliciting the data semantics from the existing data. The dataset description can help us understanding how real-life objects have been represented in the dataset. Categorical data may be identified, ranges of values, and other logical grouping of values. It could help identifying dependent statistical variables. The outcome could also let us transforming the data into other type data, such knowledge graphs with a semantic layer. 

Some reading about [Database normalisation](https://en.wikipedia.org/wiki/Database_normalization) can be useful.

## Which dataset are we using?

We use a dataset based on the Olympics game. We aim to explore whether Decision trees or random forrest may be able to predict a medal winner or not from the data. We use the 
[Olympics 124 years dataset](https://www.kaggle.com/datasets/nitishsharma01/olympics-124-years-datasettill-2020).

We are assuming all the athletes stated in this dataset have all agreed to have their name added to the data. The name of those athletes are most likely to have been in the public domain at the time of the Olympic games occured.

# Import libraries and data

In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/olympics-124-years-datasettill-2020/Athletes_summer_games.csv
/kaggle/input/olympics-124-years-datasettill-2020/Athletes_winter_games.csv
/kaggle/input/olympics-124-years-datasettill-2020/regions.csv


In [2]:
path_summer  = "/kaggle/input/olympics-124-years-datasettill-2020/Athletes_summer_games.csv"
path_winter  = "/kaggle/input/olympics-124-years-datasettill-2020/Athletes_winter_games.csv"
path_regions = "/kaggle/input/olympics-124-years-datasettill-2020/regions.csv"


We upload the data stored in the three csv files. 

## Summer data

In [3]:
summer_df = pd.read_csv(path_summer)
summer_df.dtypes

Unnamed: 0      int64
Name           object
Sex            object
Age           float64
Team           object
NOC            object
Games          object
Year            int64
Season         object
City           object
Sport          object
Event          object
Medal          object
dtype: object

In [4]:
summer_df.shape

(237673, 13)

In [5]:
summer_df.Season.unique()

array(['Summer'], dtype=object)

In [6]:
len(summer_df.NOC.unique())

233

## Winter Olympic Games data

In [7]:
winter_df = pd.read_csv(path_winter)
winter_df.dtypes

Unnamed: 0      int64
Name           object
Sex            object
Age           float64
Team           object
NOC            object
Games          object
Year            int64
Season         object
City           object
Sport          object
Event          object
Medal          object
dtype: object

In [8]:
winter_df.shape

(48564, 13)

In [9]:
winter_df.Season.unique()

array(['Winter'], dtype=object)

## Regions 

In [10]:
regions_df = pd.read_csv(path_regions)
regions_df.dtypes

Unnamed: 0     int64
NOC           object
region        object
notes         object
dtype: object

In [11]:
regions_df.head()

,Unnamed: 0,NOC,region,notes
0,0,EOR,Refugee,NaN
1,1,LBN,Lebanon,NaN
2,2,SGP,Singapore,NaN
3,3,ROC,Russia,NaN
4,4,AFG,Afghanistan,NaN


In [12]:
len(regions_df.NOC.unique())

234

## Comparisons of datasets

The number of columns is the same for both summer and winter datasets. The columns appears to have been harmonised, which is quite helpful. 

In [13]:
all(winter_df.columns.isin(summer_df.columns))

True

In [14]:
winter_df.shape[1] == summer_df.shape[1]

True

In [15]:
winter_df.columns

Index(['Unnamed: 0', 'Name', 'Sex', 'Age', 'Team', 'NOC', 'Games', 'Year',
       'Season', 'City', 'Sport', 'Event', 'Medal'],
      dtype='object')

# Merging all the data in one large dataset

The data is merge in one dataset to support analysis of the datasets, its structure and values. 

In [16]:
data_df =  pd.concat([summer_df, winter_df])
data_df.shape

(286237, 13)

In [17]:
data_df = pd.merge(data_df, regions_df, left_on = 'NOC', right_on='NOC')
data_df.dtypes

Unnamed: 0_x      int64
Name             object
Sex              object
Age             float64
Team             object
NOC              object
Games            object
Year              int64
Season           object
City             object
Sport            object
Event            object
Medal            object
Unnamed: 0_y      int64
region           object
notes            object
dtype: object

# Do all cell contain a single value?

Most of the cell contain a single value, for the exception of _Games_ column.

In [18]:
data_df.head()

,Unnamed: 0_x,Name,Sex,Age,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Unnamed: 0_y,region,notes
0,0,A Dijiang,M,24.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,45,China,NaN
1,1,A Lamusi,M,23.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,45,China,NaN
2,1072,Abudoureheman,M,22.0,China,CHN,2000 Summer,2000,Summer,Sydney,Boxing,Boxing Men's Middleweight,NaN,45,China,NaN
3,2611,Ai Linuer,M,25.0,China,CHN,2004 Summer,2004,Summer,Athina,Wrestling,"Wrestling Men's Lightweight, Greco-Roman",NaN,45,China,NaN
4,2612,Ai Yanhan,F,14.0,China,CHN,2016 Summer,2016,Summer,Rio de Janeiro,Swimming,Swimming Women's 200 metres Freestyle,NaN,45,China,NaN


### Games

A brief inspection of the data shows the column _Games_  contains several values - the year and the season.  Both values are available in the dataset in the columns _City and Season_. Counting the number of event per Olympic games (see below) suggests the values in the column _Games_ have the same values.  We have the same number of rows.  

In [19]:
columns = ['Games']
one_col = data_df.groupby(columns).count()[ 'Event'].reset_index()

print("-- columns -- \n", one_col.dtypes, sep = '')
print("--rows and columns--\n ", one_col.shape, sep = '')

-- columns -- 
Games    object
Event     int64
dtype: object
--rows and columns--
 (52, 2)


In [20]:
columns = ['Games', 'Year', 'Season']
three_cols = data_df.groupby(columns).count()[ 'Event'].reset_index() 
three_cols.dtypes
print("--columns--\n", three_cols.dtypes, sep = '')
print("--rows and columns--\n", three_cols.shape, sep = '')

--columns--
Games     object
Year       int64
Season    object
Event      int64
dtype: object
--rows and columns--
(52, 4)


In [21]:
sum(three_cols.Event == one_col.Event) == three_cols.shape[0]

True

In [22]:
sum(three_cols.Event == one_col.Event) == one_col.shape[0]

True

We remove the _Games_ column from the dataset, it contains some redundant content. Other columns provide those values.

In [23]:
col_to_drop = ['Games']
data_df = data_df.drop(columns = col_to_drop)
data_df.dtypes

Unnamed: 0_x      int64
Name             object
Sex              object
Age             float64
Team             object
NOC              object
Year              int64
Season           object
City             object
Sport            object
Event            object
Medal            object
Unnamed: 0_y      int64
region           object
notes            object
dtype: object

## Sport and events
The event columns appears to describe a specific event within a competition. Some gender appears to in the description but not in all. So no further transformation may be required. 

In [24]:
data_df.Sport.unique()

array(['Basketball', 'Judo', 'Boxing', 'Wrestling', 'Swimming',
       'Softball', 'Hockey', 'Archery', 'Triathlon', 'Football',
       'Rhythmic Gymnastics', 'Athletics', 'Badminton', 'Fencing',
       'Gymnastics', 'Volleyball', 'Baseball', 'Water Polo', 'Shooting',
       'Weightlifting', 'Cycling', 'Rowing', 'Sailing', 'Diving',
       'Modern Pentathlon', 'Art Competitions', 'Synchronized Swimming',
       'Handball', 'Canoeing', 'Table Tennis', 'Tennis', 'Taekwondo',
       'Beach Volleyball', 'Trampolining', 'Golf', 'Equestrianism',
       'Cycling Track', 'Equestrian', 'Canoe Sprint', 'Rugby Sevens',
       'Canoe Slalom', 'Trampoline Gymnastics', 'Artistic Gymnastics',
       'Artistic Swimming', '3x3 Basketball', 'Karate', 'Sport Climbing',
       'Cycling Road', 'Marathon Swimming', 'Cycling Mountain Bike',
       'Skateboarding', 'Speed Skating', 'Short Track Speed Skating',
       'Curling', 'Figure Skating', 'Snowboarding',
       'Cross Country Skiing', 'Ice Hockey', 'Fr

In [25]:
columns = ['Sport']
data_df.groupby(columns).count()['Event'].reset_index()

,Sport,Event
0,3x3 Basketball,64
1,Aeronautics,1
2,Alpine Skiing,8829
3,Alpinism,25
4,Archery,2592
...,...,...
79,Tug-Of-War,170
80,Volleyball,3692
81,Water Polo,4132
82,Weightlifting,4134


In [26]:
columns = ['Sport','Event']
data_df.groupby(columns).count()['Year'].reset_index()

,Sport,Event,Year
0,3x3 Basketball,Men Team,32
1,3x3 Basketball,Women Team,32
2,Aeronautics,Aeronautics Mixed Aeronautics,1
3,Alpine Skiing,Alpine Skiing Men's Combined,569
4,Alpine Skiing,Alpine Skiing Men's Downhill,1164
...,...,...,...
1105,Wrestling,"Wrestling Women's Flyweight, Freestyle",68
1106,Wrestling,"Wrestling Women's Heavyweight, Freestyle",64
1107,Wrestling,"Wrestling Women's Light-Heavyweight, Freestyle",18
1108,Wrestling,"Wrestling Women's Lightweight, Freestyle",67


In [27]:
len(data_df.Event.unique())

1071

In [28]:
data_df.dtypes

Unnamed: 0_x      int64
Name             object
Sex              object
Age             float64
Team             object
NOC              object
Year              int64
Season           object
City             object
Sport            object
Event            object
Medal            object
Unnamed: 0_y      int64
region           object
notes            object
dtype: object

# Do each row has a unique identifier?

The current data has yet some unique keys. Any row cannot be represented uniquely. The dataframe has an index that is unique. However, no logical grouping of the data has yet to be completed. So we explore how logical grouping of data can be made. 

## Games

From our previous discussions and evidences, we can conclude the real-life concept of a game can be uniquely identified by the composite key year and season. We explore whether other fields may be needed.  We identify the entity games as follow: 

__game__(year, season, city)

In [29]:
columns = ['Year','Season','City']
data_df.groupby(columns).count().reset_index()


,Year,Season,City,Unnamed: 0_x,Name,Sex,Age,Team,NOC,Sport,Event,Medal,Unnamed: 0_y,region,notes
0,1896,Summer,Athina,380,380,380,217,380,380,380,380,143,380,380,0
1,1900,Summer,Paris,1936,1936,1936,1146,1936,1936,1936,1936,604,1936,1936,9
2,1904,Summer,St. Louis,1301,1301,1301,1027,1301,1301,1301,1301,486,1301,1301,1
3,1906,Summer,Athina,1733,1733,1733,990,1733,1733,1733,1733,458,1733,1733,40
4,1908,Summer,London,3101,3101,3101,2452,3101,3101,3101,3101,831,3101,3101,89
5,1912,Summer,Stockholm,4040,4040,4040,3884,4040,4040,4040,4040,941,4040,4038,114
6,1920,Summer,Antwerpen,4292,4292,4292,3447,4292,4292,4292,4292,1308,4292,4292,15
7,1924,Summer,Paris,5233,5233,5233,4148,5233,5233,5233,5233,832,5233,5233,116
8,1924,Winter,Chamonix,460,460,460,403,460,460,460,460,130,460,460,7
9,1928,Summer,Amsterdam,4992,4992,4992,4119,4992,4992,4992,4992,734,4992,4992,86


We extract the columns year, season and city as one logical grouping and store them in a new dataframe. 

In [30]:
columns = ['Year', 'Season', 'City']
games = data_df.loc[:, columns].copy(deep = True)
games = games.drop_duplicates(subset=columns)
print("--columns--\n", games.dtypes, sep = '')
print("--rows and columns--\n", games.shape, sep = '')
games.sort_values('Year')

--columns--
Year       int64
Season    object
City      object
dtype: object
--rows and columns--
(53, 3)


,Year,Season,City
7074,1896,Summer,Athina
5716,1900,Summer,Paris
21672,1904,Summer,St. Louis
5733,1906,Summer,Athina
5722,1908,Summer,London
5730,1912,Summer,Stockholm
5715,1920,Summer,Antwerpen
5765,1924,Summer,Paris
19335,1924,Winter,Chamonix
5724,1928,Summer,Amsterdam


## Sports and events
The previous discussion suggests that events and sports are unique.  They could be a logical grouping too.  However, at this stage, we keep both as unique entity. 

__sports__(sport)

__event__(event)



In [31]:
sport_values = data_df.Sport.unique()
sports      = pd.DataFrame({'sport': sport_values})
print("--columns--\n", sports.dtypes, sep = '')
print("--rows and columns--\n", sports.shape, sep = '')
sports

--columns--
sport    object
dtype: object
--rows and columns--
(84, 1)


,sport
0,Basketball
1,Judo
2,Boxing
3,Wrestling
4,Swimming
...,...
79,Lacrosse
80,Jeu De Paume
81,Alpinism
82,Aeronautics


In [32]:
event_values = data_df.Event.unique()
events      = pd.DataFrame({'sport': event_values})

print("--columns--\n", events.dtypes, sep = '')
print("--rows and columns--\n", events.shape, sep = '')
events

--columns--
sport    object
dtype: object
--rows and columns--
(1071, 1)


,sport
0,Basketball Men's Basketball
1,Judo Men's Extra-Lightweight
2,Boxing Men's Middleweight
3,"Wrestling Men's Lightweight, Greco-Roman"
4,Swimming Women's 200 metres Freestyle
...,...
1066,Racquets Men's Singles
1067,Racquets Men's Doubles
1068,Motorboating Mixed B-Class (Under 60 Feet)
1069,Motorboating Mixed C-Class


## Athletes and Teams 

We group logically the name, gender, age and team name. We discover that repetitions of teams appears in our grouping. So we keep separate the athlete and the team.

In [33]:
columns = ['Name', 'Sex', 'Age', 'Team']
athlete_df = data_df.groupby(columns).count()['Sport'].reset_index()
print("--columns--\n", athlete_df.dtypes, sep = '')
print("--rows and columns--\n", athlete_df.shape, sep = '')

--columns--
Name      object
Sex       object
Age      float64
Team      object
Sport      int64
dtype: object
--rows and columns--
(193812, 5)


In [34]:
athlete_df.sort_values(['Team'])

,Name,Sex,Age,Team,Sport
65729,Harald von Musil,M,44.0,30. Februar,1
65636,Harald Fereberger,M,23.0,30. Februar,1
88807,Jos Pablo Eustaquio Manuel Francisco Escandn y...,M,44.0,A North American Team,1
88277,Jos Eustaquio Luis Francisco Escandn y Barrn,M,38.0,A North American Team,1
88614,Jos Manuel Mara del Corazn de Jess Escandn y B...,M,42.0,A North American Team,1
...,...,...,...,...,...
18181,Axel Gustaf Estlander,M,35.0,rn-2,1
26185,Carl-Oscar Girsn,M,23.0,rn-2,1
33651,Curt Magnus Wilhelm Andstn,M,30.0,rn-2,1
21487,Bertel Jusln,M,31.0,rn-2,1


In [35]:
columns = ['Name', 'Sex', 'Age']
athlete_df = data_df.groupby(columns).count()['Team'].reset_index()
print("--columns--\n", athlete_df.dtypes, sep = '')
print("--rows and columns--\n", athlete_df.shape, sep = '')

--columns--
Name     object
Sex      object
Age     float64
Team      int64
dtype: object
--rows and columns--
(192254, 4)


In [36]:
athlete_df

,Name,Sex,Age,Team
0,"Gabrielle Marie ""Gabby"" Adcock (White-)",F,25.0,1
1,Eleonora Margarida Josephina Scmitt,F,16.0,2
2,Jean Hauptmanns,M,26.0,1
3,Luis ngel Fernando de los Santos Grossi,M,23.0,1
4,Luis ngel Fernando de los Santos Grossi,M,27.0,4
...,...,...,...,...
192249,zlem Kaya,F,26.0,1
192250,zman Graud,M,33.0,1
192251,zzet Safer,M,26.0,1
192252,zzet nce,M,23.0,1


Two entities representing the real-life concepts of _athlete_ and _team_ should be created.  We propopose the following data dictionary:

__team__(team)

__athlete__(name, age, gender)

The composition of those fields appears to be composite keys.  However, we will create a numerical primary key


In [37]:
columns = ['Name', 'Sex', 'Age']
athletes = data_df.loc[:, columns].copy(deep = True)
athletes = athletes.drop_duplicates(subset = columns)
#athletes['id'] = range(1, athletes.shape[0] + 1) 
print("--columns--\n", athletes.dtypes, sep = '')
print("--rows and columns--\n", athletes.shape, sep = '')
athletes

--columns--
Name     object
Sex      object
Age     float64
dtype: object
--rows and columns--
(198610, 3)


,Name,Sex,Age
0,A Dijiang,M,24.0
1,A Lamusi,M,23.0
2,Abudoureheman,M,22.0
3,Ai Linuer,M,25.0
4,Ai Yanhan,F,14.0
...,...,...,...
286232,DOUEIHY Gabriella,F,22.0
286233,ELIAS Nacif,M,32.0
286234,FATTOUH Mahassen Hala,F,31.0
286235,HADID Noureddine,M,28.0


In [38]:
team_values = data_df.Team.unique()
teams       = pd.DataFrame({'team': team_values})

print("--columns--\n", teams.dtypes, sep = '')
print("--rows and columns--\n", teams.shape, sep = '')
teams

--columns--
team    object
dtype: object
--rows and columns--
(1196, 1)


,team
0,China
1,China-2
2,China-1
3,China-3
4,Denmark
...,...
1191,Newfoundland
1192,Kosovo
1193,South Sudan
1194,Lesotho


## Medals and National Organisational Committe
The medals and National Organisational committe are explored as two separates logical groupings.

It appears we have four types of medals with a unique value. So we could consider as categorical data.  

In [39]:
data_df['Medal'].unique()

array([nan, 'Silver', 'Bronze', 'Gold'], dtype=object)

In [40]:
data_df['Medal'] = data_df['Medal'].fillna('None')
data_df['Medal'].unique()

array(['None', 'Silver', 'Bronze', 'Gold'], dtype=object)

In [41]:
medals = pd.DataFrame({'medal': data_df['Medal'].unique()})
print("--columns--\n", medals.dtypes, sep = '')
print("--rows and columns--\n", medals.shape, sep = '')
medals


--columns--
medal    object
dtype: object
--rows and columns--
(4, 1)


,medal
0,None
1,Silver
2,Bronze
3,Gold


The National Organisation Committees can be represented with three columns - NOC, region and notes. The key appear to be composite, that is NOC and region. The NOC was the column we merge the olympic results with the regions. It is a logical grouping.

__noc__(noc, region, notes)

In [42]:
columns = ['NOC','region']
data_df.groupby(columns).count().reset_index()


,NOC,region,Unnamed: 0_x,Name,Sex,Age,Team,Year,Season,City,Sport,Event,Medal,Unnamed: 0_y,notes
0,AFG,Afghanistan,131,131,131,83,131,131,131,131,131,131,131,131,0
1,AHO,Curacao,79,79,79,78,79,79,79,79,79,79,79,79,79
2,ALB,Albania,80,80,80,80,80,80,80,80,80,80,80,80,0
3,ALG,Algeria,596,596,596,590,596,596,596,596,596,596,596,596,0
4,AND,Andorra,172,172,172,172,172,172,172,172,172,172,172,172,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,YEM,Yemen,38,38,38,38,38,38,38,38,38,38,38,38,0
226,YMD,Yemen,5,5,5,5,5,5,5,5,5,5,5,5,5
227,YUG,Serbia,2583,2583,2583,2454,2583,2583,2583,2583,2583,2583,2583,2583,2583
228,ZAM,Zambia,214,214,214,185,214,214,214,214,214,214,214,214,0


In [43]:
columns = ['NOC','region','notes']
noc = data_df.loc[:, columns].copy(deep = True)
noc = noc.drop_duplicates(subset = columns)
print("--columns--\n", noc.dtypes, sep = '')
print("--rows and columns--\n", noc.shape, sep = '')
noc


--columns--
NOC       object
region    object
notes     object
dtype: object
--rows and columns--
(233, 3)


,NOC,region,notes
0,CHN,China,NaN
5715,DEN,Denmark,NaN
9423,NED,Netherlands,NaN
15653,FIN,Finland,NaN
21169,NOR,Norway,NaN
...,...,...,...
285562,SSD,South Sudan,NaN
285567,LES,Lesotho,NaN
285635,ROC,Russia,NaN
286196,EOR,Refugee,NaN


In [44]:
noc.loc[~noc.notes.isna(), :]

,NOC,region,notes
126933,UAR,Syria,United Arab Republic
169843,YAR,Yemen,North Yemen
248002,SKN,Saint Kitts,Turks and Caicos Islands
248046,TTO,Trinidad,Trinidad and Tobago
256789,AHO,Curacao,Netherlands Antilles
262079,YMD,Yemen,South Yemen
263093,ANZ,Australia,Australasia
263179,SCG,Serbia,Serbia and Montenegro
263772,IOA,Individual Olympic Athletes,Individual Olympic Athletes
265344,YUG,Serbia,Yugoslavia


## Some logical grouping

Some columns have not been use - they are artefacts produced by data transformation.




In [45]:
data_df.dtypes

Unnamed: 0_x      int64
Name             object
Sex              object
Age             float64
Team             object
NOC              object
Year              int64
Season           object
City             object
Sport            object
Event            object
Medal            object
Unnamed: 0_y      int64
region           object
notes            object
dtype: object

The logical groupings into some entities are listed below:

__game__(year, season, city)

__sports__(sport)

__event__(event)

__team__(team)

__athlete__(name, age, gender)

__noc__(noc, region, notes)

__medal__(medal)


# Do we have any primary keys?

We had some unique identifiers, to support the data being stored in a relational database. The data can become a reference dataset to be used for groupings or certain classifiers - i.e., tree-base ML.

__game__(id(PK), year, season, city)

__sports__(id (PK), sport)

__event__(id (PK), event)

__team__(id (PK), team)

__athlete__(id (PK), name, age, gender)

__noc__(noc (PK), region, notes)

__medal__(id(PK),medal)



# What are the relationship between each logical grouping? 

## Athlete and games
We assume athletes may have participated to at least one Olympic games. Olympic games have more than one athlete... A lot more. So we can positively argue that the relationship between a games and atheletes is many to many. 

In [46]:
columns = ['Name','Age','Sex']
grouped_data = data_df.groupby(columns).count()['Year']
print("minimum attendance :" , grouped_data.min())
print("maximum attendance :" , grouped_data.max())

minimum attendance : 1
maximum attendance : 44


In [47]:
grouped_data.sort_values(ascending = False)

Name                              Age   Sex
Robert Tait McKenzie              65.0  M      44
Alfrd (Arnold-) Hajs (Guttmann-)  50.0  M      28
Miltiades Manno                   53.0  M      27
Alfred James Munnings             69.0  M      25
Wilhelm (William) Hunt Diederich  48.0  M      19
                                               ..
Irena Paweczyk (-Kowalska)        29.0  F       1
Irena Soukupov                    23.0  F       1
                                  27.0  F       1
Irena Svobodov (-Pettinari)       19.0  F       1
zzet nce                          27.0  M       1
Name: Year, Length: 192254, dtype: int64

## Athletes and teams
We surmise an athlete is part of one team and a team has many athletes. Some teams have quite a lot of athletes. So we assume a one-to-many relationship would be sufficient for this dataset.


In [48]:
data_df.sort_values(by='Name')

,Unnamed: 0_x,Name,Sex,Age,Team,NOC,Year,Season,City,Sport,Event,Medal,Unnamed: 0_y,region,notes
207460,1560,"Gabrielle Marie ""Gabby"" Adcock (White-)",F,25.0,Great Britain,GBR,2016,Summer,Rio de Janeiro,Badminton,Badminton Mixed Doubles,None,78,UK,NaN
192156,215054,Eleonora Margarida Josephina Scmitt,F,16.0,Brazil,BRA,1948,Summer,London,Swimming,Swimming Women's 100 metres Freestyle,None,33,Brazil,NaN
192157,215055,Eleonora Margarida Josephina Scmitt,F,16.0,Brazil,BRA,1948,Summer,London,Swimming,Swimming Women's 4 x 100 metres Freestyle Relay,None,33,Brazil,NaN
130443,92099,Jean Hauptmanns,M,26.0,Germany,GER,1912,Summer,Stockholm,Wrestling,"Wrestling Men's Heavyweight, Greco-Roman",None,83,Germany,NaN
196895,51334,Luis ngel Fernando de los Santos Grossi,M,23.0,Uruguay,URU,1948,Summer,London,Cycling,"Cycling Men's Team Pursuit, 4,000 metres",None,219,Uruguay,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86812,114965,zlem Kaya,F,26.0,Turkey,TUR,2016,Summer,Rio de Janeiro,Athletics,"Athletics Women's 3,000 metres Steeplechase",None,211,Turkey,NaN
86667,79870,zman Graud,M,33.0,Turkey,TUR,1972,Summer,Munich,Shooting,Shooting Mixed Skeet,None,211,Turkey,NaN
87068,207676,zzet Safer,M,26.0,Turkey,TUR,2016,Summer,Rio de Janeiro,Athletics,Athletics Men's 4 x 100 metres Relay,None,211,Turkey,NaN
86756,102956,zzet nce,M,27.0,Turkey,TUR,2008,Summer,Beijing,Weightlifting,Weightlifting Men's Light-Heavyweight,None,211,Turkey,NaN


In [49]:
columns = ['Team', 'Year']
grouped_data = data_df.groupby(columns).count()['Name']
print("minimum attendance :" , grouped_data.min())
print("maximum attendance :" , grouped_data.max())

minimum attendance : 1
maximum attendance : 936


In [50]:
grouped_data.sort_values(ascending = False)

Team           Year
United States  1992    936
               1988    886
               2020    856
Unified Team   1992    832
United States  1996    827
                      ... 
Brandenburg    1936      1
Mainz          1936      1
Brazil         1994      1
               1998      1
Konstanz       1936      1
Name: Name, Length: 5376, dtype: int64

## Sports, events and medals

We can see sports may have several events. So the relationship is one to many between sports and events. Modelling  the relationship between the medal is a bit more complex.  It appears to link a game, an event, some athtletes and a medal. We discover that a many to many between all those logical grouping would be suitable. 


In [51]:
data_df.sort_values(by=['Sport','Event','Medal','Year'])

,Unnamed: 0_x,Name,Sex,Age,Team,NOC,Year,Season,City,Sport,Event,Medal,Unnamed: 0_y,region,notes
269307,3278,DOMOVIC BULUT Dusan,M,35.0,Serbia,SRB,2020,Summer,Tokyo,3x3 Basketball,Men Team,Bronze,189,Serbia,NaN
269325,7999,MAJSTOROVIC Dejan,M,33.0,Serbia,SRB,2020,Summer,Tokyo,3x3 Basketball,Men Team,Bronze,189,Serbia,NaN
269358,10955,RATKOV Aleksandar,M,29.0,Serbia,SRB,2020,Summer,Tokyo,3x3 Basketball,Men Team,Bronze,189,Serbia,NaN
269377,13817,VASIC Mihailo,M,28.0,Serbia,SRB,2020,Summer,Tokyo,3x3 Basketball,Men Team,Bronze,189,Serbia,NaN
268385,2124,CAVARS Agnis,M,35.0,Latvia,LAT,2020,Summer,Tokyo,3x3 Basketball,Men Team,Gold,118,Latvia,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278701,90558,Adla Hanzlkov,F,22.0,Czech Republic,CZE,2016,Summer,Rio de Janeiro,Wrestling,"Wrestling Women's Middleweight, Freestyle",None,58,Czech Republic,NaN
100658,155091,Sara McMann,F,23.0,United States,USA,2004,Summer,Athina,Wrestling,"Wrestling Women's Middleweight, Freestyle",Silver,220,USA,NaN
70585,114017,Alyona Vladimirovna Kartashova,F,26.0,Russia,RUS,2008,Summer,Beijing,Wrestling,"Wrestling Women's Middleweight, Freestyle",Silver,175,Russia,NaN
1121,108583,Jing Ruixue,F,24.0,China,CHN,2012,Summer,London,Wrestling,"Wrestling Women's Middleweight, Freestyle",Silver,45,China,NaN


In [52]:
columns = ['Event','Year', 'Season', 'Sport','Name']
grouped_data = data_df.groupby(columns).count()['Medal']
print("minimum attendance :" , grouped_data.min())
print("maximum attendance :" , grouped_data.max())

minimum attendance : 1
maximum attendance : 43


In [53]:
grouped_data.sort_values(ascending = False)

Event                                                       Year  Season  Sport             Name                                   
Art Competitions Mixed Sculpturing, Unknown Event           1932  Summer  Art Competitions  Robert Tait McKenzie                       43
Art Competitions Mixed Painting, Unknown Event              1948  Summer  Art Competitions  Alfred James Munnings                      25
Art Competitions Mixed Painting, Drawings And Water Colors  1928  Summer  Art Competitions  Stanisaw Noakowski                         17
Art Competitions Mixed Painting, Unknown Event              1932  Summer  Art Competitions  Acee Blue Eagle                            17
                                                                                            Miltiades Manno                            17
                                                                                                                                       ..
Cycling Men's Road Race, Individual     

## Events and NOC
So we assume an event as one NOC. But a NOC can organise several events. However, each game has different organiser. So, we need to relate the event the game and the organisers. 

In [69]:
data_df.loc[:, ['Sport', 'Event', 'Year', 'NOC','City']].sort_values('Event')

,Sport,Event,Year,NOC,City
136184,Shooting,10m Air Pistol Men,2020,GER,Tokyo
282422,Shooting,10m Air Pistol Men,2020,MYA,Tokyo
240152,Shooting,10m Air Pistol Men,2020,PER,Tokyo
260082,Shooting,10m Air Pistol Men,2020,ISL,Tokyo
258964,Shooting,10m Air Pistol Men,2020,UKR,Tokyo
...,...,...,...,...,...
278701,Wrestling,"Wrestling Women's Middleweight, Freestyle",2016,CZE,Rio de Janeiro
223537,Wrestling,"Wrestling Women's Middleweight, Freestyle",2008,POL,Beijing
124681,Wrestling,"Wrestling Women's Middleweight, Freestyle",2016,NGR,Rio de Janeiro
270262,Wrestling,"Wrestling Women's Middleweight, Freestyle",2008,GUM,Beijing


In [68]:
columns = ['Sport','Event','Year']
grouped_data = data_df.groupby(columns).count()['NOC']
print("minimum attendance :" , grouped_data.min())
print("maximum attendance :" , grouped_data.max())

minimum attendance : 1
maximum attendance : 648


In [57]:
grouped_data.sort_values(ascending = False)

Sport             Event                                                   Year
Art Competitions  Art Competitions Mixed Painting, Unknown Event          1932    648
Football          Men Team                                                2020    344
Ice Hockey        Ice Hockey Men's Ice Hockey                             2002    312
                                                                          1998    306
Art Competitions  Art Competitions Mixed Painting, Unknown Event          1936    300
                                                                                 ... 
Sailing           Sailing Mixed 18 foot                                   1920      2
Aeronautics       Aeronautics Mixed Aeronautics                           1936      1
Art Competitions  Art Competitions Mixed Architecture                     1920      1
                  Art Competitions Mixed Music, Instrumental And Chamber  1936      1
                  Art Competitions Mixed Sculpturing, Reliefs

## The model

__game__(id(PK), year, season, city)

__game_athlete__(game_id(PK,FK), athlete_id(PK,FK))

__achievement__(game_id(FK, PK), athlete_id(FK, PK), sport_id (FK,PK), event_id(FK,PK), medal_id(FK,PK), medal_ref(PK))

__sports__(id (PK), sport)

__event__(id (PK), event, sport_id (FK))

__team__(id (PK), team)

__game_event_noc__(game_id(PK,FK), event(PK, FK), NOC(PK, FK))


__athlete__(id (PK), name, age, gender, team_id(FK))


__noc__(noc (PK), region, notes)


__medal__(id(PK),medal)
